In [2]:
# Some libraries to use later
!pip install gensim
!pip install scikit-learn matplotlib
!pip install transformers torch
!pip install datasets
!pip install -U sentence-transformers
!pip install -U accelerate
!pip install plotly

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/3f/14/e105b8ef6d324e789c1589e95cb0ab63f3e07c2216d68b1178b7c21b7d2a/torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata
  Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/0f/35/e63ba28062af0a3d688a9f128e407a1a2608544b2f480cb49bf7f4b1cbb9/sentencepiece-0.2.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Using cached sentencepiece-0.2.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 698.8 kB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 609.4 kB/s eta 0:00:0000:0100:01
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/4c/37/22ef7675bef4ffe9577b937ddca2e22791534cbbe11c30714972a91532dc/datasets-3.3.2-py3-none-any.whl.metadata
  Using cache

In [4]:
%matplotlib inline

import networkx as nx
from networkx.algorithms import community
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

ImportError: dlopen(/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libopenblas.0.dylib
  Referenced from: <68B373BB-0E19-38F4-8A21-488037FB5BFB> /Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-311-darwin.so
  Reason: tried: '/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/../../../../../../libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/../../../../../../libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/bin/../lib/libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/bin/../lib/libopenblas.0.dylib' (no such file), '/usr/local/lib/libopenblas.0.dylib' (no such file), '/usr/lib/libopenblas.0.dylib' (no such file, not in dyld cache)

Import of the dataset for the graph :

In [5]:
path_data = '/Users/sarahbouchet/Desktop/M2_TSE/S2/Web_mining/Project_documents/database_formated_for_NetworkX.graphml'

graph = nx.read_graphml(path_data)

# 1.1 Dataset exploration

Looking at the format of our graph, we have different attributes for each nodes, depending on the type of the node. We are strating by looking at how many unique values we have for the "labels" attributes. Indeed, the "labels" inform us about the type of the node as summarize in the pdf description of the data.


In [6]:

unique_labels = set(data.get("labels") for _, data in graph.nodes(data=True))

print("Distincts types of nodes using 'labels' attribut:", unique_labels)

Distincts types of nodes using 'labels' attribut: {':Event', ':PostCategory', ':Hashtag', ':Tweet', ':User'}


We are finding the 5 types of information we were expected.

In [4]:
from collections import Counter

# Compter les occurrences de chaque type de nœud
label_counts = Counter(data.get("labels") for _, data in graph.nodes(data=True))

# Trier par ordre décroissant (du plus fréquent au moins fréquent)
sorted_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)

# Afficher tous les types de nœuds
print("Number of nodes by type:")
for label, count in sorted_labels:
    print(f"{label}: {count}")

Number of nodes by type:
:Tweet: 55986
:User: 43141
:Hashtag: 10441
:Event: 34
:PostCategory: 25


As expected, the biggest number of nodes concerns the tweet, followed by the user and the Hashtag. Our dataset contains 55 986 tweets.

Now, we will focus on the information we have about the Event. We know the event's nodes have 3 attributes that are : "EventType", "id", and "trecisid".

The first thing will be to study how many type of event we have in our dataset.


In [5]:
# Extraire les valeurs uniques de "eventType" pour les nœuds de type "Event"
event_types = set(data.get("eventType") for _, data in graph.nodes(data=True) if data.get("labels") == ":Event")

# Afficher le nombre de valeurs uniques et les valeurs elles-mêmes
print(f"Number of unique values for 'eventType': {len(event_types)}")
print(f"Unique values of 'eventType': {event_types}")

Number of unique values for 'eventType': 6
Unique values of 'eventType': {'typhoon', 'flood', 'earthquake', 'bombing', 'shooting', 'wildfire'}


We thus have 6 unique types of general event that users are talking about in our dataset.

What is then relevant to know is the importance of each event in the tweets. To answer that, we can use the information given by the attributes "topic" from the "tweet"'s node.

In [6]:
from collections import Counter

# Créer une liste pour stocker les topics des tweets
tweet_topics = []

# Parcourir tous les nœuds de type "Tweet"
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet":
        # Si un attribut "topic" existe dans le tweet, l'ajouter à la liste
        topic = tweet_data.get("topic")  # Remplacer "topic" par l'attribut réel
        if topic:  # Vérifie si un topic est présent
            tweet_topics.append(topic)

# Compter les occurrences de chaque topic
topic_counts = Counter(tweet_topics)

# Trier les topics par fréquence décroissante
sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)

# Afficher les topics triés
print("Topics les plus fréquents dans les tweets :")
for topic, count in sorted_topics:
    print(f"{topic}: {count}")

Topics les plus fréquents dans les tweets :
TRECIS-CTIT-H-019: 8437
TRECIS-CTIT-H-018: 6692
TRECIS-CTIT-H-028: 2773
TRECIS-CTIT-H-027: 2709
TRECIS-CTIT-H-026: 2696
TRECIS-CTIT-H-029: 2636
TRECIS-CTIT-H-030: 2633
TRECIS-CTIT-H-031: 2567
TRECIS-CTIT-H-033: 2537
TRECIS-CTIT-H-021: 2456
TRECIS-CTIT-H-032: 2051
TRECIS-CTIT-H-034: 1993
TRECIS-CTIT-H-020: 1405
TRECIS-CTIT-H-010: 1129
TRECIS-CTIT-H-003: 1063
TRECIS-CTIT-H-011: 1025
TRECIS-CTIT-H-001: 1002
TRECIS-CTIT-H-014: 997
TRECIS-CTIT-H-005: 975
TRECIS-CTIT-H-015: 942
TRECIS-CTIT-H-006: 907
TRECIS-CTIT-H-004: 893
TRECIS-CTIT-H-025: 840
TRECIS-CTIT-H-012: 823
TRECIS-CTIT-H-022: 733
TRECIS-CTIT-H-009: 688
TRECIS-CTIT-H-013: 661
TRECIS-CTIT-H-023: 390
TRECIS-CTIT-H-002: 360
TRECIS-CTIT-H-017: 311
TRECIS-CTIT-H-007: 247
TRECIS-CTIT-H-008: 154
TRECIS-CTIT-H-024: 138
TRECIS-CTIT-H-016: 123


In [7]:
# Créer un dictionnaire pour stocker la correspondance entre "topic" et "inter" du nœud "Event"
topic_to_inter = {}

# Créer un ensemble pour les topics uniques
unique_topics = set()

# Parcourir tous les nœuds de type "Tweet" pour extraire les topics uniques
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet":
        # Extraire le topic du tweet
        topic = tweet_data.get("topic")  # Remplacer "topic" par l'attribut réel
        if topic:  # Vérifie si un topic est présent
            unique_topics.add(topic)

# Pour chaque topic unique, associer l'événement et son attribut "inter"
for topic in unique_topics:
    # Parcourir les nœuds de type "Event" pour associer le topic à l'événement correspondant
    for event_node, event_data in graph.nodes(data=True):
        if event_data.get("labels") == ":Event" and topic in event_data.get("trecisid", ""):
            # Extraire l'attribut "inter" du nœud "Event"
            id = event_data.get("id")
            # Ajouter la correspondance dans le dictionnaire
            topic_to_inter[topic] = id

# Afficher les résultats
print("Correspondance des topics uniques avec les jours ('inter') de l'événement :")
for topic, id in topic_to_inter.items():
    print(f"Topic: {topic}, id of the event : {id}")

Correspondance des topics uniques avec les jours ('inter') de l'événement :
Topic: TRECIS-CTIT-H-008, id of the event : italyEarthquakes2012
Topic: TRECIS-CTIT-H-016, id of the event : joplinTornado2011
Topic: TRECIS-CTIT-H-021, id of the event : parisAttacks2015
Topic: TRECIS-CTIT-H-030, id of the event : cycloneKenneth2019
Topic: TRECIS-CTIT-H-019, id of the event : nepalEarthquake2015
Topic: TRECIS-CTIT-H-004, id of the event : typhoonPablo2012
Topic: TRECIS-CTIT-H-006, id of the event : westTexasExplosion2013
Topic: TRECIS-CTIT-H-020, id of the event : flSchoolShooting2018
Topic: TRECIS-CTIT-H-018, id of the event : typhoonHagupit2014
Topic: TRECIS-CTIT-H-027, id of the event : shootingDallas2017
Topic: TRECIS-CTIT-H-007, id of the event : guatemalaEarthquake2012
Topic: TRECIS-CTIT-H-023, id of the event : fireAndover2019
Topic: TRECIS-CTIT-H-032, id of the event : coloradoStemShooting2019
Topic: TRECIS-CTIT-H-029, id of the event : albertaWildfires2019
Topic: TRECIS-CTIT-H-012, id

As we can see, the attribute "topic" is encoded in a way we can not clearly and explicitly understand. To translate in a more meaningful way, we can use the "trecisid" from the Event's nodes to create a connection between the information given by the data of Tweet and Event.

In [8]:
# Extraire les valeurs uniques de "eventType" pour les nœuds de type "Event"
event_types_intweets = set(data.get("topic") for _, data in graph.nodes(data=True) if data.get("labels") == ":Tweet")

# Afficher le nombre de valeurs uniques et les valeurs elles-mêmes
print(f"Number of unique values for 'event_types_intweets': {len(event_types_intweets)}")
print(f"Unique values of 'event_types_intweets': {event_types_intweets}")

Number of unique values for 'event_types_intweets': 34
Unique values of 'event_types_intweets': {'TRECIS-CTIT-H-008', 'TRECIS-CTIT-H-016', 'TRECIS-CTIT-H-021', 'TRECIS-CTIT-H-030', 'TRECIS-CTIT-H-019', 'TRECIS-CTIT-H-004', 'TRECIS-CTIT-H-006', 'TRECIS-CTIT-H-020', 'TRECIS-CTIT-H-018', 'TRECIS-CTIT-H-027', 'TRECIS-CTIT-H-007', 'TRECIS-CTIT-H-023', 'TRECIS-CTIT-H-032', 'TRECIS-CTIT-H-029', 'TRECIS-CTIT-H-012', 'TRECIS-CTIT-H-002', 'TRECIS-CTIT-H-031', 'TRECIS-CTIT-H-003', 'TRECIS-CTIT-H-033', 'TRECIS-CTIT-H-001', 'TRECIS-CTIT-H-014', 'TRECIS-CTIT-H-034', 'TRECIS-CTIT-H-009', 'TRECIS-CTIT-H-026', 'TRECIS-CTIT-H-010', 'TRECIS-CTIT-H-028', 'TRECIS-CTIT-H-015', 'TRECIS-CTIT-H-013', 'TRECIS-CTIT-H-011', 'TRECIS-CTIT-H-025', 'TRECIS-CTIT-H-017', 'TRECIS-CTIT-H-024', 'TRECIS-CTIT-H-022', 'TRECIS-CTIT-H-005'}


In [9]:
# Extraire les valeurs uniques de "eventType" pour les nœuds de type "Event"
event_types_intweets = set(data.get("trecisid") for _, data in graph.nodes(data=True) if data.get("labels") == ":Event")

# Afficher le nombre de valeurs uniques et les valeurs elles-mêmes
print(f"Number of unique values for 'event_types_intweets': {len(event_types_intweets)}")
print(f"Unique values of 'event_types_intweets': {event_types_intweets}")

Number of unique values for 'event_types_intweets': 34
Unique values of 'event_types_intweets': {'TRECIS-CTIT-H-008', 'TRECIS-CTIT-H-016', 'TRECIS-CTIT-H-021', 'TRECIS-CTIT-H-030', 'TRECIS-CTIT-H-019', 'TRECIS-CTIT-H-004', 'TRECIS-CTIT-H-006', 'TRECIS-CTIT-H-020', 'TRECIS-CTIT-H-018', 'TRECIS-CTIT-H-027', 'TRECIS-CTIT-H-007', 'TRECIS-CTIT-H-023', 'TRECIS-CTIT-H-032', 'TRECIS-CTIT-H-029', 'TRECIS-CTIT-H-012', 'TRECIS-CTIT-H-002', 'TRECIS-CTIT-H-031', 'TRECIS-CTIT-H-003', 'TRECIS-CTIT-H-033', 'TRECIS-CTIT-H-001', 'TRECIS-CTIT-H-014', 'TRECIS-CTIT-H-034', 'TRECIS-CTIT-H-009', 'TRECIS-CTIT-H-026', 'TRECIS-CTIT-H-010', 'TRECIS-CTIT-H-028', 'TRECIS-CTIT-H-015', 'TRECIS-CTIT-H-013', 'TRECIS-CTIT-H-011', 'TRECIS-CTIT-H-025', 'TRECIS-CTIT-H-017', 'TRECIS-CTIT-H-024', 'TRECIS-CTIT-H-022', 'TRECIS-CTIT-H-005'}


## 1.1.2 Work on tweets and users relations.














In [10]:
# Créer un dictionnaire pour stocker la correspondance entre Tweet et User
tweet_to_user = {}

# Parcourir tous les edges pour trouver ceux de type "POSTED"
for u, v, edge_data in graph.edges(data=True):
    if edge_data.get("label") == "POSTED":
        # Trouver le User (u) et le Tweet (v)
        user_id = graph.nodes[u].get("id")  # id du User
        tweet_id = graph.nodes[v].get("id")  # id du Tweet

        # Associer le Tweet à son User
        tweet_to_user[tweet_id] = user_id

# Afficher les 5 premiers résultats
print("Les 5 premiers Tweets et leurs Users :")
for i, (tweet, user) in enumerate(tweet_to_user.items()):
    if i == 5:
        break
    print(f"Tweet: {tweet} -> User: {user}")

Les 5 premiers Tweets et leurs Users :
Tweet: 211281973870727170 -> User: 54549327
Tweet: 211557401231495171 -> User: 78649756
Tweet: 215869691003011072 -> User: 78649756
Tweet: 211565974422425600 -> User: 210791628
Tweet: 211607187653533697 -> User: 274295451


To know which user has posted the higher number of tweets :

In [11]:
# Créer un dictionnaire pour compter les tweets par utilisateur
user_tweet_count = {}

# Parcourir tous les edges pour trouver ceux de type "POSTED"
for u, v, edge_data in graph.edges(data=True):
    if edge_data.get("label") == "POSTED":
        # Trouver le User (u) et le Tweet (v)
        user_id = graph.nodes[u].get("id")  # id du User
        tweet_id = graph.nodes[v].get("id")  # id du Tweet

        # Incrémenter le compteur de tweets pour cet utilisateur
        if user_id in user_tweet_count:
            user_tweet_count[user_id] += 1
        else:
            user_tweet_count[user_id] = 1

# Trouver l'utilisateur avec le plus grand nombre de tweets
max_user = max(user_tweet_count, key=user_tweet_count.get)
max_count = user_tweet_count[max_user]

# Afficher l'utilisateur ayant posté le plus de tweets
print(f"L'utilisateur ayant posté le plus de tweets est : {max_user} avec {max_count} tweets.")

L'utilisateur ayant posté le plus de tweets est : 44728980 avec 95 tweets.


We are now searching for the most tweeted event, using the relation "Talks_about" that link an user to an event.

In [12]:
# Créer un dictionnaire pour compter combien de fois chaque événement a été abordé
event_talk_count = {}
eventid_talk_count = {}

# Parcourir tous les edges pour trouver ceux de type "TALKS_ABOUT"
for u, v, edge_data in graph.edges(data=True):
    if edge_data.get("label") == "TALKS_ABOUT":
        # Trouver le User (u) et l'Event (v)
        user_id = graph.nodes[u].get("id")  # id du User
        event_type = graph.nodes[v].get("eventType")  # id de l'Event
        event_id = graph.nodes[v].get("id")  # id de l'Event

        # Incrémenter le compteur de l'événement
        if event_id in event_talk_count:
            event_talk_count[event_id] += 1
        else:
            event_talk_count[event_id] = 1


        if event_type in eventid_talk_count:
            eventid_talk_count[event_type] += 1
        else:
            eventid_talk_count[event_type] = 1

# Trouver l'événement le plus abordé
max_event = max(event_talk_count, key=event_talk_count.get)
max_count = event_talk_count[max_event]

max_event_id = max(eventid_talk_count, key=eventid_talk_count.get)
max_count_id = eventid_talk_count[max_event_id]

# Afficher l'événement le plus abordé et le nombre d'utilisateurs qui en ont parlé
print(f"L'événement le plus abordé est : {max_event} avec {max_count} utilisateurs qui en ont parlé.")
print(f"L'événement le plus abordé est : {max_event_id} avec {max_count_id} utilisateurs qui en ont parlé.")

L'événement le plus abordé est : nepalEarthquake2015 avec 5411 utilisateurs qui en ont parlé.
L'événement le plus abordé est : earthquake avec 8523 utilisateurs qui en ont parlé.


# 1.2 Dataset analysis at the event's scale

## 1.2.1 Creation of the database

The goal is now to start creating the dashboard, that should gives us various information at the event scale. As previously shown, we have 6 types of events and thus our analysis will be made at the scale of those 6 types.

To start with, we are linking each tweet to a particular global event in order to study the distribution of tweets related to each event from a general point of view.
For example, if a tweet is about the event "TRECIS-CTIT-H-012", we should link the tweet to the relevant general event "Bombing" because the translation of this id is "bostonBombings2013".
As a reminder, we have 6 different types of events.

In [13]:
# Dictionnaire pour associer les topics des tweets à leur EventType
topic_to_event_type = {}

# Parcourir tous les nœuds de type 'Event' pour construire la correspondance topic -> EventType
for event_node, event_data in graph.nodes(data=True):
    if event_data.get("labels") == ":Event":  # On vérifie que c'est un nœud de type Event
        event_id = event_data.get("trecisid")  # Le 'id' de l'événement (correspond au 'topic' dans le tweet)
        event_type = event_data.get("eventType")  # L'EventType explicite de l'événement
        if event_id and event_type:
            topic_to_event_type[event_id] = event_type  # Associer le 'topic' (id de l'événement) à 'eventType'

# Pour chaque tweet, associer son EventType en fonction du topic
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet":  # Vérifier que c'est un nœud de type Tweet
        tweet_topic = tweet_data.get("topic")  # Le topic du tweet, à relier à un événement
        if tweet_topic and tweet_topic in topic_to_event_type:
            event_type_for_tweet = topic_to_event_type[tweet_topic]  # Récupérer l'EventType associé au topic
            tweet_data['eventType'] = event_type_for_tweet  # Ajouter l'EventType au tweet

# Affichage des 5 premiers tweets avec leur EventType associé
counter = 0
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and 'eventType' in tweet_data:
        print(f"Tweet ID: {tweet_node}")
        print(f"Topic: {tweet_data.get('topic')}")
        print(f"Event Type: {tweet_data['eventType']}")
        print("-" * 50)

        counter += 1
        if counter == 5:
            break  # Afficher seulement les 5 premiers tweets pour vérification

Tweet ID: n582
Topic: TRECIS-CTIT-H-001
Event Type: wildfire
--------------------------------------------------
Tweet ID: n583
Topic: TRECIS-CTIT-H-001
Event Type: wildfire
--------------------------------------------------
Tweet ID: n584
Topic: TRECIS-CTIT-H-001
Event Type: wildfire
--------------------------------------------------
Tweet ID: n585
Topic: TRECIS-CTIT-H-001
Event Type: wildfire
--------------------------------------------------
Tweet ID: n586
Topic: TRECIS-CTIT-H-001
Event Type: wildfire
--------------------------------------------------


Now that the link has been created, we can count the number of tweets that concern each general event.

In [14]:
# Dictionnaire pour compter les tweets par type d'événement
event_type_count = {}

# Parcourir tous les nœuds de type 'Tweet' pour compter l'effectif par type d'événement
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and 'eventType' in tweet_data:
        event_type = tweet_data['eventType']  # Le type d'événement associé au tweet

        # Compter le nombre de tweets pour chaque type d'événement
        if event_type:
            if event_type not in event_type_count:
                event_type_count[event_type] = 0
            event_type_count[event_type] += 1

# Affichage des résultats
print("Effectif des tweets par type d'événement :")
for event_type, count in event_type_count.items():
    print(f"{event_type}: {count} tweets")

Effectif des tweets par type d'événement :
wildfire: 7826 tweets
earthquake: 13054 tweets
flood: 7808 tweets
typhoon: 13979 tweets
shooting: 9133 tweets
bombing: 4186 tweets


In [15]:
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt

# Exemple d'événement à analyser
event_type_to_analyze = 'typhoon'  # Remplace par l'événement que tu souhaites analyser

# Dictionnaire pour compter les tweets par date pour l'événement spécifié
tweet_dates = []

# Parcourir les nœuds de type 'Tweet' et associer les tweets à l'événement spécifié
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and tweet_data.get("eventType") == event_type_to_analyze:
        if 'created_at' in tweet_data:
            tweet_time = tweet_data['created_at']  # Récupérer l'heure du tweet
            tweet_dates.append(tweet_time)  # Ajouter la date du tweet à la liste

# Convertir les dates en format datetime
tweet_dates = pd.to_datetime(tweet_dates, errors='coerce')

# Retirer les valeurs invalides (NaT) générées par des erreurs de conversion
tweet_dates = tweet_dates.dropna()

# Créer une Series avec les dates des tweets
tweet_dates_series = pd.Series(tweet_dates)

# Compter le nombre de tweets par jour
tweet_counts = tweet_dates_series.groupby(tweet_dates_series.dt.date).size()

# Filtrer pour ne garder que les jours où il y a des tweets
tweet_counts = tweet_counts[tweet_counts > 0]

# Si le nombre de tweets est supérieur à 0, créer le graphique interactif
if tweet_counts.sum() > 0:
    # Créer une trace de type 'line' avec Plotly
    trace = go.Scatter(
        x=tweet_counts.index,
        y=tweet_counts.values,
        mode='lines+markers',  # Afficher à la fois les lignes et les marqueurs
        name=f"Nombre de tweets - {event_type_to_analyze}",
        line=dict(color='royalblue', width=2),
        marker=dict(size=8, color='royalblue', opacity=0.8)
    )

    # Mettre à jour la mise en page du graphique
    layout = go.Layout(
        title=f"Évolution du nombre de tweets pour l'événement '{event_type_to_analyze}'",
        title_font=dict(size=18, family='Arial, sans-serif'),
        xaxis=dict(
            title="Date",
            title_font=dict(size=14, family='Arial, sans-serif'),
            showgrid=True,
            tickangle=45,
            ticks="inside",
            tickfont=dict(size=12, family='Arial, sans-serif')
        ),
        yaxis=dict(
            title="Nombre de tweets",
            title_font=dict(size=14, family='Arial, sans-serif'),
            showgrid=True,
            tickfont=dict(size=12, family='Arial, sans-serif')
        ),
        hovermode='closest',  # Afficher des informations au survol de la souris
        plot_bgcolor='white',  # Fond blanc pour le graphique
        paper_bgcolor='rgba(0,0,0,0)',  # Fond transparent pour la page
        showlegend=True,
        xaxis_rangeslider_visible=False,  # Désactiver la barre de zoom (range slider)
        autosize=True
    )

    # Créer la figure avec les données et la mise en page
    fig = go.Figure(data=[trace], layout=layout)

    # Afficher le graphique
    fig.show()
else:
    print(f"Aucun tweet pour l'événement '{event_type_to_analyze}'")

## 1.2.3. Graph visualisation

We are creating a summary graph, where all the general event's temporal distribution are represented, in order to compare the temporality, the importance and the evolution of each of them.

In [16]:
# Extraction des labels uniques
unique_labels = set(data.get("labels") for _, data in graph.nodes(data=True))

# Créer un dictionnaire pour stocker la correspondance entre "topic" et "inter" du nœud "Event"
topic_to_inter = {}

# Créer un ensemble pour les topics uniques
unique_topics = set()

# Parcourir tous les nœuds de type "Tweet" pour extraire les topics uniques
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet":
        # Extraire le topic du tweet
        topic = tweet_data.get("topic")  # Remplacer "topic" par l'attribut réel
        if topic:  # Vérifie si un topic est présent
            unique_topics.add(topic)

# Pour chaque topic unique, associer l'événement et son attribut "inter"
for topic in unique_topics:
    # Parcourir les nœuds de type "Event" pour associer le topic à l'événement correspondant
    for event_node, event_data in graph.nodes(data=True):
        if event_data.get("labels") == ":Event" and topic in event_data.get("trecisid", ""):
            # Extraire l'attribut "inter" du nœud "Event"
            id = event_data.get("id")
            # Ajouter la correspondance dans le dictionnaire
            topic_to_inter[topic] = id

# Créer un dictionnaire pour stocker la correspondance entre Tweet et User
tweet_to_user = {}

# Parcourir tous les edges pour trouver ceux de type "POSTED"
for u, v, edge_data in graph.edges(data=True):
    if edge_data.get("label") == "POSTED":
        # Trouver le User (u) et le Tweet (v)
        user_id = graph.nodes[u].get("id")  # id du User
        tweet_id = graph.nodes[v].get("id")  # id du Tweet

        # Associer le Tweet à son User
        tweet_to_user[tweet_id] = user_id

# Dictionnaire pour associer les topics des tweets à leur EventType
topic_to_event_type = {}

# Parcourir tous les nœuds de type 'Event' pour construire la correspondance topic -> EventType
for event_node, event_data in graph.nodes(data=True):
    if event_data.get("labels") == ":Event":  # On vérifie que c'est un nœud de type Event
        event_id = event_data.get("trecisid")  # Le 'id' de l'événement (correspond au 'topic' dans le tweet)
        event_type = event_data.get("eventType")  # L'EventType explicite de l'événement
        if event_id and event_type:
            topic_to_event_type[event_id] = event_type  # Associer le 'topic' (id de l'événement) à 'eventType'

# Pour chaque tweet, associer son EventType en fonction du topic
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet":  # Vérifier que c'est un nœud de type Tweet
        tweet_topic = tweet_data.get("topic")  # Le topic du tweet, à relier à un événement
        if tweet_topic and tweet_topic in topic_to_event_type:
            event_type_for_tweet = topic_to_event_type[tweet_topic]  # Récupérer l'EventType associé au topic
            tweet_data['eventType'] = event_type_for_tweet  # Ajouter l'EventType au tweet

# Dictionnaire pour compter les tweets par type d'événement
event_type_count = {}

# Parcourir tous les nœuds de type 'Tweet' pour compter l'effectif par type d'événement
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and 'eventType' in tweet_data:
        event_type = tweet_data['eventType']  # Le type d'événement associé au tweet

        # Compter le nombre de tweets pour chaque type d'événement
        if event_type:
            if event_type not in event_type_count:
                event_type_count[event_type] = 0
            event_type_count[event_type] += 1



In [18]:
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt

# Liste des types d'événements
event_types = ['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake', 'flood']
event_colors = {
    'typhoon': 'royalblue',
    'shooting': 'darkorange',
    'wildfire': 'green',
    'bombing': 'red',
    'earthquake': 'purple',
    'flood': 'cyan'
}

# Créer une liste pour stocker les traces
traces = []

# Parcourir tous les types d'événements
for event_type in event_types:
    tweet_dates = []

    # Parcourir les nœuds de type 'Tweet' et associer les tweets à l'événement spécifié
    for tweet_node, tweet_data in graph.nodes(data=True):
        if tweet_data.get("labels") == ":Tweet" and tweet_data.get("eventType") == event_type:
            if 'created_at' in tweet_data:
                tweet_time = tweet_data['created_at']  # Récupérer l'heure du tweet
                tweet_dates.append(tweet_time)  # Ajouter la date du tweet à la liste

    # Convertir les dates en format datetime
    tweet_dates = pd.to_datetime(tweet_dates, errors='coerce')

    # Retirer les valeurs invalides (NaT) générées par des erreurs de conversion
    tweet_dates = tweet_dates.dropna()

    # Créer une Series avec les dates des tweets
    tweet_dates_series = pd.Series(tweet_dates)

    # Compter le nombre de tweets par jour
    tweet_counts = tweet_dates_series.groupby(tweet_dates_series.dt.date).size()

    # Créer une série de toutes les dates possibles entre la première et la dernière date
    #all_dates = pd.date_range(start=tweet_counts.index.min(), end=tweet_counts.index.max(), freq='D')

    # Réindexer tweet_counts pour inclure toutes les dates, en mettant les dates manquantes à zéro
    #tweet_counts_full = tweet_counts.reindex(all_dates, fill_value=0)
    tweet_counts_full = tweet_counts

    # Si le nombre de tweets est supérieur à 0, ajouter la trace
    if tweet_counts_full.sum() > 0:
        trace = go.Scatter(
            x=tweet_counts_full.index,
            y=tweet_counts_full.values,
            mode='lines+markers',  # Afficher à la fois les lignes et les marqueurs
            name=f"Nombre de tweets - {event_type}",
            line=dict(color=event_colors[event_type], width=2),
            marker=dict(size=8, color=event_colors[event_type], opacity=0.5)
        )
        traces.append(trace)

# Mettre à jour la mise en page du graphique
layout = go.Layout(
    title="Évolution du nombre de tweets par type d'événement",
    title_font=dict(size=18, family='Arial, sans-serif'),
    xaxis=dict(
        title="Date",
        title_font=dict(size=14, family='Arial, sans-serif'),
        showgrid=True,
        tickangle=45,
        ticks="inside",
        tickfont=dict(size=12, family='Arial, sans-serif')
    ),
    yaxis=dict(
        title="Nombre de tweets",
        title_font=dict(size=14, family='Arial, sans-serif'),
        showgrid=True,
        tickfont=dict(size=12, family='Arial, sans-serif')
    ),
    hovermode='closest',  # Afficher des informations au survol de la souris
    plot_bgcolor='white',  # Fond blanc pour le graphique
    paper_bgcolor='rgba(0,0,0,0)',  # Fond transparent pour la page
    showlegend=True,  # Afficher la légende
    xaxis_rangeslider_visible=False,  # Désactiver la barre de zoom (range slider)
    autosize=True
)

# Créer la figure avec les données et la mise en page
fig = go.Figure(data=traces, layout=layout)

# Afficher le graphique
fig.show()

## 1.2.4. Creation of the dashboard

We are creating a table where all the necessary informations are presented for the creation of the graph :

In [19]:
# Liste des types d'événements
event_types = ['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake', 'flood']

# Stocker les données
data = []

# Parcourir tous les types d'événements
for event_type in event_types:
    tweet_dates = []

    # Parcourir les nœuds de type 'Tweet' et associer les tweets à l'événement spécifié
    for tweet_node, tweet_data in graph.nodes(data=True):
        if tweet_data.get("labels") == ":Tweet" and tweet_data.get("eventType") == event_type:
            if 'created_at' in tweet_data:
                tweet_time = tweet_data['created_at']
                tweet_dates.append(tweet_time)  # Ajouter la date du tweet

    # Convertir les dates en format datetime
    tweet_dates = pd.to_datetime(tweet_dates, errors='coerce')
    tweet_dates = tweet_dates.dropna()  # Supprimer les valeurs NaT

    # Compter le nombre de tweets par date
    # Convert tweet_dates to a Series before using groupby and size
    tweet_dates_series = pd.Series(tweet_dates)
    tweet_counts = tweet_dates_series.groupby(tweet_dates_series.dt.date).size()

    # Ajouter les données à la table
    for date, count in tweet_counts.items():
        data.append({'date': date, 'Event_Type': event_type, 'num_tweets_perday': count})

# Création du DataFrame
df_tweets_time_series = pd.DataFrame(data)

# Vérification des premières lignes
(df_tweets_time_series.head())

,date,Event_Type,num_tweets_perday
0,2011-05-23,typhoon,10
1,2011-05-24,typhoon,12
2,2011-05-25,typhoon,27
3,2011-05-26,typhoon,9
4,2011-05-27,typhoon,8


In [20]:
import pandas as pd

# Liste des types d'événements et leurs couleurs
event_types = ['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake', 'flood']
event_colors = {
    'typhoon': 'royalblue',
    'shooting': 'darkorange',
    'wildfire': 'green',
    'bombing': 'red',
    'earthquake': 'purple',
    'flood': 'cyan'
}

# Liste pour stocker les statistiques pour chaque type d'événement
event_stats = []

# Liste pour stocker les données des relations IS_ABOUT
data_is_about = []

# Parcourir les relations IS_ABOUT (user - event)
for u, v, edge_data in graph.edges(data=True):
    if edge_data.get("label") == "IS_ABOUT":  # Vérifier le bon label
        data_is_about.append({"user": u, "event": v, "label": "IS_ABOUT"})

# Créer un DataFrame des relations IS_ABOUT
df_is_about = pd.DataFrame(data_is_about)

# Liste pour stocker les données des événements
event_data = []

# Parcourir les nœuds pour trouver ceux de type "Event"
for node, data in graph.nodes(data=True):
    if data.get("labels") == ":Event":  # Vérifie si c'est un Event
        event_data.append({"event": node, "event_id": data.get("id"), "eventType": data.get("eventType")})

# Créer le DataFrame des événements
df_events = pd.DataFrame(event_data)

# Joindre les DataFrames sur la colonne "event"
df_combined = pd.merge(df_is_about, df_events, on='event', how='left')

# Regrouper par "eventType" et compter le nombre unique d'utilisateurs
user_counts_by_event = df_combined.groupby('eventType')['user'].nunique().reset_index()
sous_event_id_counts = df_combined.groupby(['eventType'])['event_id'].nunique().reset_index(name='unique_event_id_count')

# Renommer la colonne pour correspondre à l'output attendu
user_counts_by_event = user_counts_by_event.rename(columns={"user": "Number of Users"})
sous_event_id_counts = sous_event_id_counts.rename(columns={"user": "Number of unique event in the category"})

# Afficher les résultats


# Calcul des statistiques pour chaque type d'événement
for event_type in event_types:
    tweet_dates = []
    user_ids = []

    # Sélectionner les utilisateurs et tweets associés à cet événement
    event_df = df_combined[df_combined['eventType'] == event_type]

    # Parcourir les nœuds de type 'Tweet' et associer les tweets avec l'événement
    for tweet_node, tweet_data in graph.nodes(data=True):
        if tweet_data.get("labels") == ":Tweet" and tweet_data.get("eventType") == event_type:
            if 'created_at' in tweet_data:
                tweet_time = tweet_data['created_at']  # Récupérer le timestamp du tweet
                user_id = tweet_data['id']  # ID de l'utilisateur
                tweet_dates.append(tweet_time)  # Ajouter la date du tweet à la liste
                user_ids.append(user_id)  # Ajouter l'ID de l'utilisateur à la liste

    # Convertir les dates des tweets en datetime
    tweet_dates = pd.to_datetime(tweet_dates, errors='coerce')

    # Supprimer les valeurs invalides (NaT)
    tweet_dates = tweet_dates.dropna()

    # Créer une série des dates des tweets
    tweet_dates_series = pd.Series(tweet_dates)

    # Compter le nombre de tweets par jour
    tweet_counts = tweet_dates_series.groupby(tweet_dates_series.dt.date).size()

    # Calcul des statistiques pour l'événement
    num_tweets = tweet_counts.sum()  # Nombre total de tweets pour cet événement
    num_users = user_counts_by_event[user_counts_by_event['eventType'] == event_type]['Number of Users'].values[0]  # Nombre d'utilisateurs uniques
    num_subevent = sous_event_id_counts[sous_event_id_counts['eventType'] == event_type]['unique_event_id_count'].values[0]
    first_tweet_date = tweet_dates.min()  # Date du premier tweet
    last_tweet_date = tweet_dates.max()  # Date du dernier tweet
    avg_tweet_freq = (last_tweet_date - first_tweet_date).days / num_tweets if num_tweets > 0 else 0  # Fréquence moyenne des tweets en jours



    # Ajouter les statistiques à la liste
    event_stats.append({
        'Event_Type': event_type,
        'Number_of_Tweets_perEvent': num_tweets,
        'Number_of_Users_perEvent': num_users,
        'Nb_of_sub_event': num_subevent,
        'First_Tweet_Date': first_tweet_date,
        'Last_Tweet_Date': last_tweet_date,
        'Avg_Tweet_Frequency': avg_tweet_freq
    })

# Créer un DataFrame à partir des statistiques
event_stats_df = pd.DataFrame(event_stats)

# Afficher le DataFrame des statistiques
(event_stats_df)




,Event_Type,Number_of_Tweets_perEvent,Number_of_Users_perEvent,Nb_of_sub_event,First_Tweet_Date,Last_Tweet_Date,Avg_Tweet_Frequency
0,typhoon,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311
1,shooting,9133,5559,5,2013-11-01 00:00:00+00:00,2019-05-08 00:00:00+00:00,0.220519
2,wildfire,7826,5778,4,2012-06-09 00:00:00+00:00,2019-06-03 00:00:00+00:00,0.325837
3,bombing,4186,2783,3,2013-04-15 00:00:00+00:00,2015-11-14 00:00:00+00:00,0.225275
4,earthquake,13054,9380,8,2012-05-20 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.194193
5,flood,7808,4255,7,2011-02-02 00:00:00+00:00,2019-04-24 00:00:00+00:00,0.384606


In [21]:
df_complete = df_tweets_time_series.merge(event_stats_df, on=['Event_Type'], how='left')
df_complete.head()

,date,Event_Type,num_tweets_perday,Number_of_Tweets_perEvent,Number_of_Users_perEvent,Nb_of_sub_event,First_Tweet_Date,Last_Tweet_Date,Avg_Tweet_Frequency
0,2011-05-23,typhoon,10,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311
1,2011-05-24,typhoon,12,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311
2,2011-05-25,typhoon,27,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311
3,2011-05-26,typhoon,9,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311
4,2011-05-27,typhoon,8,13979,8913,6,2011-05-23 00:00:00+00:00,2019-04-29 00:00:00+00:00,0.207311


In [22]:
#!pip install jupyter_bokeh
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

In [23]:
choice = pn.widgets.ToggleGroup(
    name='CHOICE',
    options=['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake'],
    value=['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake'],
    button_type='success')
choice

idf = df_complete.interactive()

In [24]:
idf.head()

BokehModel(combine_events=True, render_bundle={'docs_json': {'3d4aa11e-7525-4491-8c2a-c3999c30e0b8': {'version…

In [25]:
idf.columns

BokehModel(combine_events=True, render_bundle={'docs_json': {'dee42de1-037a-4fa5-a34d-9e7e65aab6db': {'version…

In [27]:
choice = pn.widgets.ToggleGroup(
    name='CHOICE',
    options=['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake'],
    value=['typhoon', 'shooting', 'wildfire', 'bombing', 'earthquake'],
    button_type='success')
choice

idf = df_complete.interactive()

ipipeline = (
    idf[idf.Event_Type.isin(choice)]  # Filtrer par event_type en fonction du choix utilisateur
    .groupby(['date', 'Event_Type'])  # Garder la granularité par date et event_type
    .agg({'num_tweets_perday' : 'unique',
        'Number_of_Tweets_perEvent': 'unique',
        'Number_of_Users_perEvent': 'unique',
        'Nb_of_sub_event': 'unique',
        'First_Tweet_Date': 'min',
        'Last_Tweet_Date': 'max',
        'Avg_Tweet_Frequency': 'unique'
    })
    .reset_index()
)

ipipeline.head()

BokehModel(combine_events=True, render_bundle={'docs_json': {'6c10ba80-9702-4a0a-a73f-01fc42332a72': {'version…

In [30]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme="default")
itable

BokehModel(combine_events=True, render_bundle={'docs_json': {'75ca18bc-822e-4515-820a-1379959570de': {'version…

In [31]:


import panel as pn

# Nouvelle palette avec 2 couleurs ajoutées
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", "#6a5acd", "#ff6347"]  # Ajout de deux nouvelles couleurs

# Utilisation de pn.pane.HTML pour créer des blocs colorés avec les nouvelles couleurs
pn.Row(
    pn.pane.HTML(f'<div style="background-color:{PALETTE[0]}; height:50px; width:100%"></div>'),
    pn.pane.HTML(f'<div style="background-color:{PALETTE[1]}; height:50px; width:100%"></div>'),
    pn.pane.HTML(f'<div style="background-color:{PALETTE[2]}; height:50px; width:100%"></div>'),
    pn.pane.HTML(f'<div style="background-color:{PALETTE[3]}; height:50px; width:100%"></div>'),  # Nouveau bloc avec la 4ème couleur
    pn.pane.HTML(f'<div style="background-color:{PALETTE[4]}; height:50px; width:100%"></div>')   # Nouveau bloc avec la 5ème couleur
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'d460314b-fa73-4238-868a-eab382d0b588': {'version…

In [32]:
hvplot = ipipeline.hvplot(x='date',y='num_tweets_perday', by='Event_Type')
hvplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'b86a4912-7afa-4951-85fe-7f4a442bdd57': {'version…

Here is the table and the graph together as a Dashboard, at the scale of each event type : 

In [ ]:
import panel as pn
import hvplot.pandas  # Pour les graphes interactifs
import pandas as pd


# Function to dynamically update the graph and table
@pn.depends(choice, watch=True)
def update_graph_and_table(choice):

    # Creating the dynamic graph
    hvplot_graph = ipipeline.hvplot(
        x='date', y='num_tweets_perday', by='Event_Type',
        kind='line', title='Nombre de Tweets par Jour et par Type d\'Événement',
        xlabel='Date', ylabel='Nombre de Tweets', line_width=2, responsive=True, height=500
    )

    # Creating the dynamic table with Tabulator
    itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme='default')

    # Returning the graph and table in a panel, wrapping hvplot_graph with pn.pane.HoloViews
    return pn.Column(
        pn.pane.HoloViews(hvplot_graph),  # Wrap hvplot_graph here
        itable  # Display the table
    )

# ... (rest of your dashboard code)

# Créer le layout Panel avec le widget et la mise à jour du graphique et de la table
dashboard = pn.Column(

    update_graph_and_table(choice)  # Graphique et table mis à jour dynamiquement
)

# Afficher le dashboard
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'ad75ff54-f2a5-45a7-bd33-6e2e75d2231f': {'version…

# 3. TWEET-level vector semantics

In the following section, we will implement the functions to compute ththeese tweet-level vector semantics based on the word enbeddings method. At the tweet-level, we compute a global tweet representation by averaging the word embeddings for all the words in the tweet.

As an illustration, we manually choose two tweets and visualize them. The relevant part of the tweet is the content which is accessible in the "text" attribut.

These functions compute the representation of each word and the average representation of each word within a wholde tweet, to study the tweet-level vector semantics.

FONCTION TRES LONGUE QUI ARRIVE AVEC WORD TO VEC CA TOURNE LONGTEMPS

In [ ]:
# Some libraries to use later
!pip install gensim
!pip install scikit-learn matplotlib
!pip install transformers torch
!pip install datasets
!pip install -U sentence-transformers
!pip install -U accelerate
!pip install hvplot
!pip install datasets
!pip install datasets


# import datasets
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.manifold import TSNE
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import nltk

import torch
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer


import pandas as pd
import hvplot.pandas
import panel as pn



ImportError: dlopen(/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libopenblas.0.dylib
  Referenced from: <68B373BB-0E19-38F4-8A21-488037FB5BFB> /Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-311-darwin.so
  Reason: tried: '/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/../../../../../../libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/lib/python3.11/site-packages/scipy/sparse/linalg/_isolve/../../../../../../libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/bin/../lib/libopenblas.0.dylib' (no such file), '/Users/sarahbouchet/anaconda3/bin/../lib/libopenblas.0.dylib' (no such file), '/usr/local/lib/libopenblas.0.dylib' (no such file), '/usr/lib/libopenblas.0.dylib' (no such file, not in dyld cache)

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/4c/37/22ef7675bef4ffe9577b937ddca2e22791534cbbe11c30714972a91532dc/datasets-3.3.2-py3-none-any.whl.metadata
  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-19.0.1.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for tqdm>=4.66.3 from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl.metadata
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Obtaining dep

These functions compute the representation of each word and the average representation of each word within a wholde tweet, to study the tweet-level vector semantics.

In [ ]:
def get_word_embeddings():
  # Download and load a pre-trained Word2Vec model
  nltk.download('punkt_tab')
  model = api.load("word2vec-google-news-300")
  return model

def get_tweet_word_embedding(doc, model):
    def preprocess_text(text):
      return word_tokenize(text.lower())

    words = preprocess_text(doc)
    word_vectors = []
    for word in words:
        if word in model:
            word_vectors.append(model[word])

    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)

    document_embedding = np.mean(word_vectors, axis=0)
    return document_embedding
w2v_model = get_word_embeddings()

# This function converts each tweet to a vector :

tweet_embeddings = []

# Iterate over the nodes of type 'Tweet'
for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and 'text' in tweet_data:  # Assuming 'text' key holds tweet content
        tweet_text = tweet_data['text']  # Extract the text of the tweet
        tweet_vector = get_tweet_word_embedding(tweet_text, w2v_model)  # Get the embedding
        tweet_embeddings.append(tweet_vector)  # Append the vector representation

# You now have `tweet_embeddings` which is a list of vectors representing the tweets

# Now, let's convert them into a DataFrame or further processing
tweet_embeddings_df = pd.DataFrame(tweet_embeddings)  # Create DataFrame for analysis


# Perform t-SNE to reduce dimensions to 2D
tsne_w2v = TSNE(perplexity=15, n_components=2, init='pca', n_iter=1000, random_state=42)
w2v_2d = tsne_w2v.fit_transform(np.array(tweet_embeddings))  # Transpose to work with word vectors


Visual representation just for two tweets (without the aggregation by Topic Event ==> a discuter) : 

In [ ]:
def plot_doc_points(q1, q2, vis_2d):
    # Plot representation of each word
    plt.figure(figsize=(12, 8))

    # Access the first element of q1 (index 0)
    x, y = vis_2d[q1[0], 0], vis_2d[q1[0], 1]
    plt.scatter(x, y, color='blue', s=50)
    plt.text(x + 0.1, y + 0.1, "doc1", fontsize=10) # Change label to "doc1"

    x, y = vis_2d[q2[0], 0], vis_2d[q2[0], 1]  # Access the first element of q2 (index 0)
    plt.scatter(x, y, color='red', s=50)
    plt.text(x + 0.1, y + 0.1, "doc2", fontsize=10) # Change label to "doc2"


    plt.title("Documents Representations", fontsize=14)
    plt.xlabel("t-SNE X", fontsize=12)
    plt.ylabel("t-SNE Y", fontsize=12)
    plt.grid(True)
    plt.show()

#we are plotting tweet at index 0 and tweet at index 20
q1 = 0
q2 = 20

plot_doc_points([q1], [q2], w2v_2d)

To be more precise, we can directly group the tweets given the event type they talk about, and apply the same methodology on the tweet group. The visualisation will be at the event type scale now.

In [ ]:
# creation of the data : 
event_types = []

for tweet_node, tweet_data in graph.nodes(data=True):
    if tweet_data.get("labels") == ":Tweet" and 'eventType' in tweet_data:
        event_type = tweet_data['eventType']  # Récupérer uniquement le type d'événement
        event_types.append(event_type)

event_types_df = pd.DataFrame({'eventType': event_types})
event_types_df.head()

len(event_types_df)

len(tweet_embeddings_df)
tweet_embeddings_df = pd.DataFrame(tweet_embeddings)
tweet_embeddings_df['eventType'] = event_types

tweet_embeddings_df.head()

In [ ]:
### Visualisation by Event type (si on aggrége avant)
w2v_2d_df = pd.DataFrame(w2v_2d)
w2v_2d_df['eventType'] = event_types

w2v_2d_df.head()

# Créer un DataFrame avec les coordonnées 2D et les types d'événements
w2v_2d_df = pd.DataFrame(w2v_2d, columns=["x", "y"])
w2v_2d_df["eventType"] = event_types

# Création du widget de filtrage
event_filter = pn.widgets.Select(name="Event Type", options=list(w2v_2d_df["eventType"].unique()))

# Fonction de mise à jour du graphique
@pn.depends(event_filter)
def update_plot(eventType):
    filtered_df = w2v_2d_df[w2v_2d_df["eventType"] == eventType]
    return filtered_df.hvplot.scatter(
        x="x", y="y",
        title=f"Tweet Embeddings pour {eventType}",
        xlabel="t-SNE X", ylabel="t-SNE Y",
        size=50, hover_cols=["eventType"], color="blue"
    )

# Création du dashboard interactif
dashboard = pn.Column(event_filter, update_plot)

# Affichage interactif dans le notebook avec .servable() pour que le panel soit rendu correctement
dashboard.servable()
